<a href="https://colab.research.google.com/github/Anthony-Ng-20/City_Prioritisation_For_Food_Delivery/blob/main/City_Prioritisation_For_Food_Delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Which UK cities to enter for a food delivery company

This short insight investigates the top cities to enter in the U.K. for a hypothetical food delivery company. 

Suggestions are split by within England, Scotland, Wales and Northern Ireland.



# Summary of Findings
(For the convenience of the reader, visulisations are placed here but they need to be run after the code section)
*   find the top 10 locations to enter in England, 5 Wales, 5 Scotland, 5 Northern Ireland
*   closest city to 



# Method of estimation

There are multiple factors for a company to consider when picking the right location to enter the food delivery business.
Some factors include addressable market size, availability of drivers, and regulations. 

For this analysis, we mainly focus on estimating the market size since this would be the main factor affecting the potential economic gain for the company. 

We estimated the market size of different cities within the U.K. by leveraging open-source population data, takeaway spending surveyed by KPMG, and heuristics.

Specifically, we calculated estimations for market size by the following formula:
Age group population * % of the population that uses 3rd party food delivery app* approx. spending on takeaway per person per year



*   Location granularity - there are different ways to segment population area, which is the best segmentations to follow? Higher population area could be prioritise, areas can be filtered by minimum population size
*   Behaviours are hard to predict and there are limited data points. For an existing delivery company, we might be able to 





# Other lessons learnt



*   Nomis datasets are useful free resources for market sizing in the U.K.. However, it might be a bit difficult to work with since areas within the U.K. might be split differently across different datasets.


*   There are different models to calculate distance between two points on Earth.

# Further works

Enhance the average takeaway spending, (this could be with a regression model, K nearest neighbour)

Additional assumptions on the behaviour of the target populaion.

*   % of population that is tech-savvy (i.e. knows how to use food delivery apps/ internet)
*   % of population that uses food delivery apps
*   % of food delivery spend out of the total food takeaway pending




# Data Sources

[KPMG average takeaway spend data](https://home.kpmg/uk/en/home/insights/2021/07/how-restaurant-operators-can-take-advantage-of-the-takeaway-boom.html) 

[Population data from NOMIS](https://www.nomisweb.co.uk/query/select/getdatasetbytheme.asp?opt=3&theme=&subgrp=)

# Code & Working

## Set-up

In [ ]:
#Install packages 
!pip install keplergl # for geo-visulisations
!pip install requests # for API

     |████████████████████████████████| 9.7 MB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 46.4 MB/s 
     |████████████████████████████████| 15.4 MB 59.1 MB/s 
     |████████████████████████████████| 6.3 MB 58.5 MB/s 
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922343 sha256=35ece8f716cfda136c7d73b7795060516d55e746d748938bd8c4bf3716fa7490
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [ ]:
#Import packages 
from google.colab import output
output.enable_custom_widget_manager() # enable 3rd party widget 

#from keplergl import KeplerGl # KeplerGl visualisation tool
from geopy.geocoders import Nominatim

import pandas as pd
import re 
import string
import requests
import geopy.distance
import numpy as np

pd.options.display.float_format = '{:20,.2f}'.format

## Load data and inspect rows

In [ ]:
def read_fixed_format(path, n_rows):
  df = pd.read_excel(path, 
                        sheet_name = "Data", 
                        header = None,
                        names = ["local_authority",
                                  "all_ages",
                                  "age_18",
                                  "age_19",
                                  "aged_20_24",
                                  "aged_25_29",
                                  "aged_30_34",
                                  "aged_35_39",
                                  "aged_40_44",
                                  "aged_45_49",
                                  "aged_50_54",
                                  "aged_55_59",
                                  "aged_60_64",
                                  "aged_65_69",
                                  "aged_70_74",
                                  "aged_75_79",
                                  "aged_80_84",
                                  "aged_85+"
                                  ],
                        usecols = "A:R",
                        skiprows = 7,
                        nrows = n_rows)
  return df

In [ ]:
# providing raw url to download csv from github
avg_spend_csv        = '/content/drive/MyDrive/Personal_Project/UK_City_Average_Spend_On_Takeaways.csv' # average takeaway spend from KPMG research

# Population data of England, Wales, Scotland, Northern Ireland
major_city_csv       = '/content/drive/MyDrive/Personal_Project/Major_cities_towns_england_wales_28_OCT_2019.csv' # from national statistic in 2020

england_xlsx       = '/content/drive/MyDrive/Personal_Project/nomis_england_pop_estimate_2020_data.xlsx'
scotland_xlsx      = '/content/drive/MyDrive/Personal_Project/nomis_scotland_pop_estimate_2020_data.xlsx'
wales_xlsx         = '/content/drive/MyDrive/Personal_Project/nomis_wales_pop_estimate_2020_data.xlsx'
n_ireland_xlsx     = '/content/drive/MyDrive/Personal_Project/nomis_northern_ireland_pop_estimate_2020_data.xlsx'
pop_den_2019       = '/content/drive/MyDrive/Personal_Project/uk_population_density_mid_2019.xlsx'


spending   = pd.read_csv(avg_spend_csv,dtype={
                     'City': str,
                     'Latitude': float,
                     'Longitude': float,
                     'Average_Spend_2019': int,
                     'Average_Spend_2021': int,
                 })
major_city = pd.read_csv(major_city_csv,dtype={
                     'major town and city': str,
                     'All Ages': float,
                     'Aged 0 to 15': int,
                     'Aged 16 to 17': int,
                     'Aged 18 to 24': int,
                     'Aged 18 to 21': int,
                     'Aged 25 to 49': int,
                     'Aged 50 to 64': int,
                     'Aged 65+': int,
                     'Age 65': int
                 }
                         )


england = read_fixed_format(england_xlsx,309)
scotland = read_fixed_format(scotland_xlsx,32)
wales = read_fixed_format(wales_xlsx,22)
n_ireland = read_fixed_format(n_ireland_xlsx,11)

pop_density = pd.read_excel(pop_den_2019, 
                        sheet_name = "data", 
                        header = None,
                        names = ["la_name",
                                  "median_age",
                                  "population_density",
                                  "population_density_decile"
                                  ],
                        usecols = "B:E",
                        skiprows = 2,
                        nrows = 368)

In [ ]:
spending.head()

,City,Latitude,Longitude,Average_Spend_2019,Average_Spend_2021,Unnamed: 5
0,Belfast,54.607868,-5.926437,369,383,103.79%
1,Birmingham,52.489471,-1.898575,365,628,172.05%
2,Brighton,50.827778,-0.152778,365,431,118.08%
3,Bristol,51.454514,-2.587910,318,598,188.05%
4,Cardiff,51.481583,-3.179090,192,431,224.48%


In [ ]:
major_city.head()

,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65
0,Barnsley,99642.0,18235,2040,6935,3401,33166,20216,19050,1060
1,Basildon,118145.0,26878,2864,8915,4720,40997,21173,17318,1012
2,Basingstoke,115111.0,23793,2465,8376,4281,41671,21732,17074,949
3,Bath,105730.0,15576,1971,21848,12713,31911,16996,17428,857
4,Bedford,93767.0,19663,2376,7449,3889,31695,17197,15387,885


In [ ]:
england.head()

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+
0,Darlington,107402,1111,860,5322,6341,6669,6562,6412,7069,7776,7731,6785,5818,5992,4132,3226,2963
1,County Durham,533149,5452,7548,37066,34101,30856,31195,28450,32439,38266,39370,34845,31052,31144,21669,15151,12566
2,Hartlepool,93836,979,942,5162,6122,6163,5671,5055,5635,6594,7003,6031,5172,5051,3154,2582,2412
3,Middlesbrough,141285,1579,1861,10741,11124,10063,8639,7294,7920,8380,9136,8205,6772,6107,4246,3274,2815
4,Northumberland,323820,3237,2574,14253,16286,17121,17553,17743,20041,23477,26299,25067,22490,22858,15486,10638,9896


In [ ]:
scotland.head()

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+
0,Aberdeen City,229060,2116,2824,17791,20808,21689,18275,14810,13963,13881,14204,12687,10552,9818,6344,5066,4831
1,Aberdeenshire,260780,2568,2296,11843,12971,14716,16872,16977,18739,20072,19907,17326,15347,14730,9567,6670,5792
2,Angus,115820,1145,1193,5762,5984,6077,6575,6387,7364,8536,9112,8224,7477,7786,5391,3809,3578
3,Argyll and Bute,85430,834,859,4701,4243,3764,4331,4404,5260,6518,7192,6678,6147,6185,4364,3054,2592
4,Clackmannanshire,51290,522,516,2814,2960,2809,2900,2991,3555,4149,4041,3447,3062,3058,2113,1296,1047


In [ ]:
wales.head()

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+
0,Isle of Anglesey,70440,679,594,3309,3599,3756,3539,3537,4247,4831,5190,5181,4829,5089,3752,2566,2403
1,Gwynedd,125171,1379,1949,10176,8932,7209,6146,5834,7091,8052,8640,7987,7263,7664,5611,3848,4176
2,Conwy,118184,1123,1066,5610,5817,5936,5876,5661,6899,8403,9126,8545,8091,8714,6364,4597,5172
3,Denbighshire,96664,1036,907,4967,5495,5215,4560,4859,5870,6945,7317,6477,6239,6691,4717,3200,2670
4,Flintshire,156847,1655,1540,8102,9254,9758,9301,8681,10375,11603,11432,9653,8936,9520,6644,4481,3713


In [ ]:
n_ireland.head()

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+
0,Antrim and Newtownabbey,143756,1715,1582,8137,8819,9190,9472,9297,9636,10050,9850,8192,6902,6288,4966,3368,2997
1,"Armagh City, Banbridge and Craigavon",217232,2529,2125,11227,14047,14935,14822,14091,14260,14776,14199,11931,9624,8918,7001,4630,3994
2,Belfast,342560,3845,4934,28402,25916,26198,24069,20923,19954,21167,21534,18598,14164,12162,10098,7391,7435
3,Causeway Coast and Glens,144943,1780,1687,8970,8182,8522,8589,8568,9671,10694,10273,8878,7640,6987,5648,3594,3221
4,Derry City and Strabane,151109,1956,1799,9458,9331,10081,9441,9255,10204,11008,10293,8305,7152,6326,4565,3011,2374


In [ ]:
pop_density.head()

,la_name,median_age,population_density,population_density_decile
0,Tower Hamlets,31.6,16237,10
1,Islington,31.9,16164,10
2,Hackney,33.3,14796,10
3,Kensington and Chelsea,39.9,13011,10
4,Westminster,36.3,12444,10


## Determine the closest city with takeaway spending data 

In [ ]:
# Function to return longtitude, latitude co-ordinates of the cities contained in the dataframe

def location(df):
  # Get list of cities
  city_name = df['local_authority']
  city_country = city_name + ', United Kingdom'

  # Get longtitude and latitude for cities through API

  geolocator = Nominatim(user_agent="PersonalProject")
  city_location = []

  for city in city_country:
    city_location.append(geolocator.geocode(city))

  city_location

  return city_location

In [ ]:
# function to return information on the closest city with average takeaway spending, given the co-ordinates of the city/local authority in question
def closest_city(co_ord):
  temp_ref_city_list = []
  temp_dist_list = []
  closest_city_info =[]

  for i in range(len(spending)):
    temp_ref_city_list.append(
        [
        spending['City'][i],
        (spending['Latitude'][i],spending['Longitude'][i]),
        spending['Average_Spend_2021'][i],
        spending['Average_Spend_2019'][i]
        ]
    )

  # Determine the distance between the city in question and other cities with takeaway spending information
  for i in range(len(temp_ref_city_list)):
    temp_dist_list.append(geopy.distance.vincenty(co_ord, temp_ref_city_list[i][1]).km)


  index_min = np.argmin(temp_dist_list)

  # Return the name, co-ordinates, the distance ,and average takeaway spending 

  closest_city_info = [
                      temp_ref_city_list[index_min][0], # city name
                      temp_ref_city_list[index_min][1], # city co-ordinates
                      temp_ref_city_list[index_min][2], # takeaway spending 2021
                      temp_ref_city_list[index_min][3], # takeaway spending 2019
                      temp_dist_list[index_min], # shortest distance value

  ]

  return closest_city_info

In [ ]:
# function to add approximated takeaway spending per person to the dataframe that contains local authority informaton
def append_avg_takeaway_spend(df):
  
  # Temp list to store avg spend
  closest_city_spend_2021 = []
  closest_city_spend_2019 = []
  proxy_city = []
  temp_location = []
  
  # temp list to extract location info
  temp_location = location(df)

  # store average spend to temp lists
  for i in range(len(temp_location)):
      closest_city_spend_2021.append(
          closest_city(temp_location[i][1])[2]
      )

      closest_city_spend_2019.append(
          closest_city(temp_location[i][1])[3]
      )

      proxy_city.append(
          closest_city(temp_location[i][1])[0]
      )

  # append new columns to dataframe

  df['estimated_avg_spend_2021'] = closest_city_spend_2021
  df['estimated_avg_spend_2019'] = closest_city_spend_2019
  df['proxy_city'] = proxy_city
 
  return df

## Finding out the cities/ areas to enter 

In [ ]:
# Append the approximated avg spend 

england_list = append_avg_takeaway_spend(england)
scotland_list = append_avg_takeaway_spend(scotland)
wales_list = append_avg_takeaway_spend(wales)

In [ ]:
# District names in Northern Ireland are problematic when looking up their  
# co-ordinates via the API

n_ireland_co_ord = location(n_ireland)
n_ireland_co_ord

[Location(The White House, 28-34, Whitehouse Park, Rushpark, Merville Garden Village, Newtownabbey, County Antrim, Northern Ireland, BT37 9SQ, United Kingdom, (54.65739255, -5.9072795495719195, 0.0)),
 None,
 Location(Belfast, County Antrim, Northern Ireland, BT1 5GS, United Kingdom, (54.596391, -5.9301829, 0.0)),
 Location(Causeway Coast and Glens Heritage Trust, Main Street, Armoy, County Antrim, Northern Ireland, BT53 8RH, United Kingdom, (55.130595400000004, -6.327745858099967, 0.0)),
 Location(Derry City and Strabane District Council, Derry Road, Strabane, County Tyrone, Northern Ireland, BT82 8AQ, United Kingdom, (54.8351485, -7.457994790640386, 0.0)),
 Location(Fermanagh and Omagh District Council, Northern Ireland, United Kingdom, (54.4413243, -7.7567616226758105, 0.0)),
 None,
 None,
 None,
 None,
 None]

In [ ]:
# Belfast is the only city in the Northern Ireland that we have takeaway spending data
# We will use this city as the proxy for all districts in Northern Ireland

n_ireland_list = n_ireland
n_ireland_list['estimated_avg_spend_2021'] = int(spending[spending['City']=='Belfast']['Average_Spend_2021'])
n_ireland_list['estimated_avg_spend_2019'] = int(spending[spending['City']=='Belfast']['Average_Spend_2019'])

## Top locations to enter in England

In [ ]:
england_list

# Food delivery app usage as found by KPMG 
app_usage_18_to_34 = 0.54
app_usage_55_plus = 0.20
app_usage_avg = 0.34

england_list['aged_18_to_34'] = (england_list['age_18']
                                 + england_list['age_19']
                                 + england_list['aged_20_24']
                                 + england_list['aged_25_29']
                                 + england_list['aged_30_34'])

england_list['aged_35_to_54'] = (england_list['aged_35_39']
                                 + england_list['aged_40_44']
                                 + england_list['aged_45_49']
                                 + england_list['aged_50_54'])

england_list['aged_55_to_64'] = (england_list['aged_55_59']
                                 + england_list['aged_60_64'])

approx_market_size_2021 = (
                           england_list['aged_18_to_34']*app_usage_18_to_34
                           + england_list['aged_35_to_54']*app_usage_avg
                           + england_list['aged_55_to_64']*app_usage_55_plus
                           )*england_list['estimated_avg_spend_2021']

approx_market_size_2019 = (
                           england_list['aged_18_to_34']*app_usage_18_to_34
                           + england_list['aged_35_to_54']*app_usage_avg
                           + england_list['aged_55_to_64']*app_usage_55_plus
                          )*england_list['estimated_avg_spend_2019']

england_list['approx_market_size_2021'] = pd.to_numeric(approx_market_size_2021,
                                                        downcast='integer')
                                           

england_list['approx_market_size_2019'] = pd.to_numeric(approx_market_size_2019,
                                                        downcast='integer')

In [ ]:
england_list = england_list.sort_values(by='approx_market_size_2021', ascending = False)



In [ ]:
england_list.reset_index(drop=True, inplace=True)

england_list.head(20)

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+,estimated_avg_spend_2021,estimated_avg_spend_2019,proxy_city,aged_18_to_34,aged_35_to_54,aged_55_to_64,approx_market_size_2021,approx_market_size_2019,la_name_x,median_age_x,population_density_x,population_density_decile_x,la_name_y,median_age_y,population_density_y,population_density_decile_y,la_name,median_age,population_density,population_density_decile
0,Birmingham,1140525,15027,19848,104991,97954,83554,74911,68219,65245,65426,60245,49184,41424,36517,27821,21633,22017,628,365,Birmingham,321374,273801,109429,"181,190,622.80","105,309,836.50",Birmingham,32.60,"4,261.00",9.00,Birmingham,32.60,"4,261.00",9.00,Birmingham,32.60,"4,261.00",9.00
1,Manchester,555741,6555,11503,67658,64937,53258,41921,32353,28195,28639,25275,20018,15897,13498,9077,6762,6323,696,361,Manchester,203911,131108,45293,"113,968,092.96","59,112,760.86",Manchester,30.10,"4,766.00",10.00,Manchester,30.10,"4,766.00",10.00,Manchester,30.10,"4,766.00",10.00
2,Nottingham,337098,4932,13639,49831,32935,23986,20153,17669,17262,17849,16679,13753,11550,10124,6755,5238,5458,1097,299,Nottingham,125323,72933,30432,"108,118,169.88","29,468,853.96",Nottingham,29.70,"4,439.00",10.00,Nottingham,29.70,"4,439.00",10.00,Nottingham,29.70,"4,439.00",10.00
3,Leicester,354036,4565,6905,39691,31848,26023,23849,20801,19325,18687,18428,16243,13624,11047,7274,5727,5930,1097,299,Nottingham,109032,82662,34671,"103,026,466.32","28,081,051.44",Leicester,31.60,"4,852.00",10.00,Leicester,31.60,"4,852.00",10.00,Leicester,31.60,"4,852.00",10.00
4,Buckinghamshire,547060,6229,4410,24377,28346,31093,35165,37188,38913,40356,38342,31723,26525,26920,20175,15179,15315,781,690,London,94455,151622,70065,"91,041,310.58","80,433,424.20",Buckinghamshire,42.20,348.00,4.00,Buckinghamshire,42.20,348.00,4.00,Buckinghamshire,42.20,348.00,4.00
5,Leeds,798786,9003,14696,79880,69664,54378,52024,46160,46317,47988,45933,38378,33075,33455,23032,17942,16280,434,336,Leeds,227621,192489,84311,"89,067,129.20","68,955,196.80",Leeds,35.30,"1,437.00",7.00,Leeds,35.30,"1,437.00",7.00,Leeds,35.30,"1,437.00",7.00
6,North Northamptonshire,350448,3706,2950,16409,19775,21889,23269,21979,24246,25772,24420,20072,18370,18720,12597,8267,7653,1097,299,Nottingham,64729,95266,44492,"83,638,022.50","22,796,507.50",NaN,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,nan,nan
7,"Bristol, City of",465866,4693,8233,52041,50930,43695,33537,27227,24609,24272,22954,18994,16344,15564,10987,8354,9025,598,318,Bristol,159592,109645,41948,"78,845,450.84","41,927,848.44","Bristol, City of",32.40,"4,213.00",9.00,"Bristol, City of",32.40,"4,213.00",9.00,"Bristol, City of",32.40,"4,213.00",9.00
8,Newham,355266,3893,4066,26349,35424,39825,35328,25304,20980,19839,16521,13259,9398,6993,4717,3622,3234,781,690,London,109557,101451,29780,"77,795,503.72","68,730,982.80",Newham,32.30,"9,809.00",10.00,Newham,32.30,"9,809.00",10.00,Newham,32.30,"9,809.00",10.00
9,Lambeth,321813,2813,2620,21880,41109,41024,31448,22759,19473,18895,17584,12519,8883,6914,4697,3824,3732,781,690,London,109446,92575,30103,"75,442,210.14","66,651,888.60",Lambeth,33.30,"12,075.00",10.00,Lambeth,33.30,"12,075.00",10.00,Lambeth,33.30,"12,075.00",10.00


England represents the largest market to the U.K. since it contains 56 million people out 67 million people in the U.K. (in 2021). This is about 83% of the total population.

Above is the top 20 districts to enter in England. Most notably in the list is that London is missing. Since we are working with local authorities data, London is actually broken by the different councils within London. For example, Newham is actually a London Borough.

Another factor to consider is also population density. The size of district varies significantly. It is more sensible to enter into an area where there is a relatively high population density for a number of reasons. Higher population density could mean you have more reliable supplies of drivers and restuarants. It could also mean deliveries would be shorter in duration because eaters might live closer to the restuarants and distances between subsequent deliveries might be shorter which helps the driver to be more efficient.

In [ ]:
# London Boroughs

'Barking and Dagenham'

In [ ]:
england_list_w_density = pd.merge(england_list, 
                        pop_density,
                        left_on='local_authority', 
                        right_on='la_name',
                        how='left')

england_list.head(20)

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+,estimated_avg_spend_2021,estimated_avg_spend_2019,proxy_city,aged_18_to_34,aged_35_to_54,aged_55_to_64,approx_market_size_2021,approx_market_size_2019,la_name_x,median_age_x,population_density_x,population_density_decile_x,la_name_y,median_age_y,population_density_y,population_density_decile_y,la_name,median_age,population_density,population_density_decile
0,Birmingham,1140525,15027,19848,104991,97954,83554,74911,68219,65245,65426,60245,49184,41424,36517,27821,21633,22017,628,365,Birmingham,321374,273801,109429,1.811906e+08,1.053098e+08,Birmingham,32.6,4261.0,9.0,Birmingham,32.6,4261.0,9.0,Birmingham,32.6,4261.0,9.0
1,Manchester,555741,6555,11503,67658,64937,53258,41921,32353,28195,28639,25275,20018,15897,13498,9077,6762,6323,696,361,Manchester,203911,131108,45293,1.139681e+08,5.911276e+07,Manchester,30.1,4766.0,10.0,Manchester,30.1,4766.0,10.0,Manchester,30.1,4766.0,10.0
2,Nottingham,337098,4932,13639,49831,32935,23986,20153,17669,17262,17849,16679,13753,11550,10124,6755,5238,5458,1097,299,Nottingham,125323,72933,30432,1.081182e+08,2.946885e+07,Nottingham,29.7,4439.0,10.0,Nottingham,29.7,4439.0,10.0,Nottingham,29.7,4439.0,10.0
3,Leicester,354036,4565,6905,39691,31848,26023,23849,20801,19325,18687,18428,16243,13624,11047,7274,5727,5930,1097,299,Nottingham,109032,82662,34671,1.030265e+08,2.808105e+07,Leicester,31.6,4852.0,10.0,Leicester,31.6,4852.0,10.0,Leicester,31.6,4852.0,10.0
4,Buckinghamshire,547060,6229,4410,24377,28346,31093,35165,37188,38913,40356,38342,31723,26525,26920,20175,15179,15315,781,690,London,94455,151622,70065,9.104131e+07,8.043342e+07,Buckinghamshire,42.2,348.0,4.0,Buckinghamshire,42.2,348.0,4.0,Buckinghamshire,42.2,348.0,4.0
5,Leeds,798786,9003,14696,79880,69664,54378,52024,46160,46317,47988,45933,38378,33075,33455,23032,17942,16280,434,336,Leeds,227621,192489,84311,8.906713e+07,6.895520e+07,Leeds,35.3,1437.0,7.0,Leeds,35.3,1437.0,7.0,Leeds,35.3,1437.0,7.0
6,North Northamptonshire,350448,3706,2950,16409,19775,21889,23269,21979,24246,25772,24420,20072,18370,18720,12597,8267,7653,1097,299,Nottingham,64729,95266,44492,8.363802e+07,2.279651e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Bristol, City of",465866,4693,8233,52041,50930,43695,33537,27227,24609,24272,22954,18994,16344,15564,10987,8354,9025,598,318,Bristol,159592,109645,41948,7.884545e+07,4.192785e+07,"Bristol, City of",32.4,4213.0,9.0,"Bristol, City of",32.4,4213.0,9.0,"Bristol, City of",32.4,4213.0,9.0
8,Newham,355266,3893,4066,26349,35424,39825,35328,25304,20980,19839,16521,13259,9398,6993,4717,3622,3234,781,690,London,109557,101451,29780,7.779550e+07,6.873098e+07,Newham,32.3,9809.0,10.0,Newham,32.3,9809.0,10.0,Newham,32.3,9809.0,10.0
9,Lambeth,321813,2813,2620,21880,41109,41024,31448,22759,19473,18895,17584,12519,8883,6914,4697,3824,3732,781,690,London,109446,92575,30103,7.544221e+07,6.665189e+07,Lambeth,33.3,12075.0,10.0,Lambeth,33.3,12075.0,10.0,Lambeth,33.3,12075.0,10.0


In [ ]:
england_list[england_list['proxy_city']=='London'].head(27)

,local_authority,all_ages,age_18,age_19,aged_20_24,aged_25_29,aged_30_34,aged_35_39,aged_40_44,aged_45_49,aged_50_54,aged_55_59,aged_60_64,aged_65_69,aged_70_74,aged_75_79,aged_80_84,aged_85+,estimated_avg_spend_2021,estimated_avg_spend_2019,proxy_city,aged_18_to_34,aged_35_to_54,aged_55_to_64,approx_market_size_2021,approx_market_size_2019,la_name_x,median_age_x,population_density_x,population_density_decile_x,la_name_y,median_age_y,population_density_y,population_density_decile_y,la_name,median_age,population_density,population_density_decile
4,Buckinghamshire,547060,6229,4410,24377,28346,31093,35165,37188,38913,40356,38342,31723,26525,26920,20175,15179,15315,781,690,London,94455,151622,70065,91041310.58,80433424.2,Buckinghamshire,42.2,348.0,4.0,Buckinghamshire,42.2,348.0,4.0,Buckinghamshire,42.2,348.0,4.0
8,Newham,355266,3893,4066,26349,35424,39825,35328,25304,20980,19839,16521,13259,9398,6993,4717,3622,3234,781,690,London,109557,101451,29780,77795503.72,68730982.8,Newham,32.3,9809.0,10.0,Newham,32.3,9809.0,10.0,Newham,32.3,9809.0,10.0
9,Lambeth,321813,2813,2620,21880,41109,41024,31448,22759,19473,18895,17584,12519,8883,6914,4697,3824,3732,781,690,London,109446,92575,30103,75442210.14,66651888.6,Lambeth,33.3,12075.0,10.0,Lambeth,33.3,12075.0,10.0,Lambeth,33.3,12075.0,10.0
10,Wandsworth,329735,2524,2621,20863,40510,39392,35379,25160,20650,18562,15289,11799,9244,8383,5753,4515,4105,781,690,London,105910,99751,27088,75385509.54,66601794.6,Wandsworth,34.0,9696.0,10.0,Wandsworth,34.0,9696.0,10.0,Wandsworth,34.0,9696.0,10.0
11,Barnet,399007,4302,3201,21331,27693,31639,31618,30051,27701,26443,23093,19076,15723,14752,10781,7854,9060,781,690,London,88166,115813,42169,74522910.66,65839703.4,Barnet,37.4,4550.0,10.0,Barnet,37.4,4550.0,10.0,Barnet,37.4,4550.0,10.0
12,Southwark,320017,2903,3136,22067,35306,38521,29876,23151,20006,19812,17794,13601,8946,7043,4851,3612,3486,781,690,London,101933,92845,31395,72547183.72,64094182.8,Southwark,33.6,10994.0,10.0,Southwark,33.6,10994.0,10.0,Southwark,33.6,10994.0,10.0
13,Croydon,388563,4470,3450,19436,24890,29584,29780,28123,25883,26949,26229,20412,16035,13652,9588,7427,7346,781,690,London,81830,110735,46641,71200880.30,62904747.0,Croydon,37.7,4497.0,10.0,Croydon,37.7,4497.0,10.0,Croydon,37.7,4497.0,10.0
17,Lewisham,305309,2856,2566,17937,26656,31045,29598,25521,21241,19446,17790,13023,9055,7245,5045,4045,3964,781,690,London,81060,95806,30813,64439560.24,56931237.6,Lewisham,35.2,8738.0,10.0,Lewisham,35.2,8738.0,10.0,Lewisham,35.2,8738.0,10.0
18,Ealing,340341,3740,3281,18362,23266,26542,26697,25887,24490,23208,20350,16651,13977,11292,7928,6340,6124,781,690,London,75191,100282,37001,64119490.82,56648461.8,Ealing,37.1,6104.0,10.0,Ealing,37.1,6104.0,10.0,Ealing,37.1,6104.0,10.0
19,Brent,327753,3620,3669,21548,24934,26707,26720,23012,20965,20381,19311,16982,12878,10344,7289,5833,5679,781,690,London,80478,91078,36293,63794610.44,56361435.6,Brent,35.7,7669.0,10.0,Brent,35.7,7669.0,10.0,Brent,35.7,7669.0,10.0


## Create the priority list


*   Maxmium average spend = city population * expected avg. spend per person for the city
*   Rank the city by the average spend


In [ ]:
major_city['estimated_avg_spend_2021'] = closest_city_spend_2021
major_city['estimated_avg_spend_2019'] = closest_city_spend_2019

In [ ]:
major_city['target_population'] = major_city['Aged 25 to 49'] + major_city['Aged 50 to 64'] + major_city['Age 65'] 

In [ ]:
major_city['market_size_2021'] = major_city['target_population'] * major_city['estimated_avg_spend_2021'] 
major_city['market_size_2019'] = major_city['target_population'] * major_city['estimated_avg_spend_2019'] 

In [ ]:
major_city

,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65,estimated_avg_spend_2021,estimated_avg_spend_2019,target_population,market_size_2021,market_size_2019
0,Barnsley,99642.0,18235,2040,6935,3401,33166,20216,19050,1060,395,558,54442,21504590,30378636
1,Basildon,118145.0,26878,2864,8915,4720,40997,21173,17318,1012,781,690,63182,49345142,43595580
2,Basingstoke,115111.0,23793,2465,8376,4281,41671,21732,17074,949,531,362,64352,34170912,23295424
3,Bath,105730.0,15576,1971,21848,12713,31911,16996,17428,857,598,318,49764,29758872,15824952
4,Bedford,93767.0,19663,2376,7449,3889,31695,17197,15387,885,781,690,49777,38875837,34346130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Worthing,115319.0,19976,2329,7126,3812,34518,23991,27379,1272,431,365,59781,25765611,21820065
108,York,164934.0,24988,2858,27811,17292,55435,27290,26552,1420,434,336,84145,36518930,28272720
109,Cardiff,356425.0,65865,7148,52783,29385,123170,56677,50782,2987,431,192,182834,78801454,35104128
110,Newport,137563.0,28624,3187,11275,6029,46496,25466,22515,1165,431,192,73127,31517737,14040384


In [ ]:
major_city.sort_values(by='market_size_2021', ascending=False)[0:10]


,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65,estimated_avg_spend_2021,estimated_avg_spend_2019,target_population,market_size_2021,market_size_2019
53,London,8960924.0,1843714,193239,733350,362411,3623682,1475795,1091144,69085,781,690,5168562,4036646922,3566307780
6,Birmingham,1159888.0,262997,29873,141968,79121,398290,177827,148933,8799,628,365,584916,367327248,213494340
50,Leicester,415584.0,87318,9542,55321,32442,141577,65685,56141,3602,1097,299,210864,231317808,63048336
56,Manchester,566896.0,113812,11876,87444,46160,223161,76717,53886,3560,696,361,303438,211192848,109541118
14,Bristol,580199.0,107805,11011,72537,38330,217817,88296,82733,4532,598,318,310645,185765710,98785110
64,Nottingham,320536.0,59077,6375,65938,43066,107255,45621,36270,2295,1097,299,155171,170222587,46396129
29,Derby,264430.0,55104,6244,25606,14648,86527,47575,43374,2256,1097,299,136358,149584726,40771042
26,Coventry,388793.0,74619,8188,55960,30298,142067,56388,51571,2878,628,365,201333,126437124,73486545
49,Leeds,516298.0,101168,10386,84350,46586,179936,76271,64187,3714,434,336,259921,112805714,87333456
79,Sheffield,557039.0,101044,11613,77158,43064,188426,92409,86389,4623,395,558,285458,112755910,159285564


In [ ]:
major_city.sort_values(by='market_size_2019', ascending=False)[0:10]

,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65,estimated_avg_spend_2021,estimated_avg_spend_2019,target_population,market_size_2021,market_size_2019
53,London,8960924.0,1843714,193239,733350,362411,3623682,1475795,1091144,69085,781,690,5168562,4036646922,3566307780
6,Birmingham,1159888.0,262997,29873,141968,79121,398290,177827,148933,8799,628,365,584916,367327248,213494340
79,Sheffield,557039.0,101044,11613,77158,43064,188426,92409,86389,4623,395,558,285458,112755910,159285564
52,Liverpool,589774.0,105162,11111,76361,44860,202609,104553,89978,5556,346,377,312718,108200428,117894686
56,Manchester,566896.0,113812,11876,87444,46160,223161,76717,53886,3560,696,361,303438,211192848,109541118
14,Bristol,580199.0,107805,11011,72537,38330,217817,88296,82733,4532,598,318,310645,185765710,98785110
73,Reading,257653.0,53944,5598,24542,13802,90728,45561,37280,2129,781,690,138418,108104458,95508420
49,Leeds,516298.0,101168,10386,84350,46586,179936,76271,64187,3714,434,336,259921,112805714,87333456
54,Luton,222043.0,54405,5528,17736,9754,80081,35906,28387,1646,781,690,117633,91871373,81166770
26,Coventry,388793.0,74619,8188,55960,30298,142067,56388,51571,2878,628,365,201333,126437124,73486545


In [ ]:
major_city.sort_values(by='All Ages', ascending=False)[0:15]

,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65,estimated_avg_spend_2021,estimated_avg_spend_2019,target_population,market_size_2021,market_size_2019
53,London,8960924.0,1843714,193239,733350,362411,3623682,1475795,1091144,69085,781,690,5168562,4036646922,3566307780
6,Birmingham,1159888.0,262997,29873,141968,79121,398290,177827,148933,8799,628,365,584916,367327248,213494340
52,Liverpool,589774.0,105162,11111,76361,44860,202609,104553,89978,5556,346,377,312718,108200428,117894686
14,Bristol,580199.0,107805,11011,72537,38330,217817,88296,82733,4532,598,318,310645,185765710,98785110
56,Manchester,566896.0,113812,11876,87444,46160,223161,76717,53886,3560,696,361,303438,211192848,109541118
79,Sheffield,557039.0,101044,11613,77158,43064,188426,92409,86389,4623,395,558,285458,112755910,159285564
49,Leeds,516298.0,101168,10386,84350,46586,179936,76271,64187,3714,434,336,259921,112805714,87333456
50,Leicester,415584.0,87318,9542,55321,32442,141577,65685,56141,3602,1097,299,210864,231317808,63048336
26,Coventry,388793.0,74619,8188,55960,30298,142067,56388,51571,2878,628,365,201333,126437124,73486545
12,Bradford,358573.0,92084,10563,34366,19604,121823,56477,43260,2893,434,336,181193,78637762,60880848


In [ ]:
major_city[major_city['major town and city']=='Oldham']

,major town and city,All Ages,Aged 0 to 15,Aged 16 to 17,Aged 18 to 24,Aged 18 to 21,Aged 25 to 49,Aged 50 to 64,Aged 65+,Age 65,estimated_avg_spend_2021,estimated_avg_spend_2019,target_population,market_size_2021,market_size_2019
66,Oldham,121852.0,32007,3639,11548,6525,41749,18856,14053,943,696,361,61548,42837408,22218828


## Takeaways

London and Birmingham being the most populated cities in the U.K. are the obvious top choices to launch a takeaway business in the U.K.

In [ ]:
index_min = np.argmin(temp_dist_list)

In [ ]:
index_min
temp_dist_list[index_min]

18.888252568151657

In [ ]:


# Function to find the closest city to a given co-ordinate of a city

def cloest_ref_city(city_location):

  # Offload reference city names and co-ordinates for distance calculations
  temp_ref_city_coord = []
  temp_ref_city_name = []

  for i in range(len(spending)):
    temp_ref_city_coord.append((spending['Latitude'][i],spending['Longitude'][i]))
    temp_ref_city_name.append(spending['City'][i])

for i in range(len(ref_city_list)):
  temp_dist_list.append([ref_city_list[i][0], geopy.distance.vincenty(city_location[0][1], ref_city_list[i][1]).km ])

In [ ]:
# Function to determine which reference city is the closest, given the name of closest_city 
# List of to-dos:
# 1) get full list of name -> add long and lat to the temp df -> compare long and lat -> calculate distance to the closest reference city by function  -> do the matchign


# Get list of cities
city_name = major_city['major_town_and_city']
city_country = city_name + ', United Kingdom'

# Get longtitude and latitude for cities through API

geolocator = Nominatim(user_agent="PersonalProject")
city_location = []

for city in city_country:
  city_location.append(geolocator.geocode(city))

# Calculate all the distance between the city to the reference cities

## Combine t



AttributeError: ignored

In [ ]:
city_location[0][1]

(53.5527719, -1.4827755)

In [ ]:
temp_dist_list

[['Belfast', 313.5812837983668],
 ['Birmingham', 121.57570710892323],
 ['Brighton', 316.5452222712779],
 ['Bristol', 245.24284987540761],
 ['Cardiff', 257.6232910159793],
 ['Edinburgh', 288.879393429476],
 ['Glasgow', 312.7834851109451],
 ['Leeds', 27.997980712058432],
 ['Liverpool', 101.06927151236353],
 ['London', 245.45764502510968],
 ['Manchester', 51.11180180616133],
 ['Newcastle', 157.56563340709533],
 ['Norwich', 212.6197568051233],
 ['Nottingham', 70.66436355155084],
 ['Plymouth', 397.8891442068403],
 ['Sheffield', 18.888252568151657],
 ['Southampton', 294.1480665202492]]

In [ ]:
len(spending)

ref_city_list =[]

for record in range(len(spending)):
  temp = [spending['City'][record],(spending['Latitude'][record],spending['Longitude'][record])]
  ref_city_list.append(temp)

In [ ]:
ref_city_list

[['Belfast', (54.607868, -5.926437)],
 ['Birmingham', (52.489470999999995, -1.898575)],
 ['Brighton', (50.827778, -0.152778)],
 ['Bristol', (51.454514, -2.58791)],
 ['Cardiff', (51.481583, -3.17909)],
 ['Edinburgh', (55.953251, -3.1882669999999997)],
 ['Glasgow', (55.860916, -4.2514330000000005)],
 ['Leeds', (53.801277, -1.548567)],
 ['Liverpool', (53.400002, -2.983333)],
 ['London', (51.509865000000005, -0.118092)],
 ['Manchester', (53.483959, -2.244644)],
 ['Newcastle', (54.966667, -1.6)],
 ['Norwich', (52.630886, 1.297355)],
 ['Nottingham', (52.950001, -1.15)],
 ['Plymouth', (50.376289, -4.143841)],
 ['Sheffield', (53.383331000000005, -1.466667)],
 ['Southampton', (50.909698, -1.4043510000000001)]]

In [ ]:
temp[1]

(54.607868, -5.926437)

In [ ]:
city_location[2]

Location(Basingstoke, Hampshire, South East England, England, United Kingdom, (51.262826, -1.0861976, 0.0))

In [ ]:
list_test =[]

geolocator = Nominatim(user_agent="PersonalProject")
location = geolocator.geocode("London, United Kingdom")

list_test.append(location)

In [ ]:
list_test[0][1][0]

51.5073219

In [ ]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

In [ ]:
geopy.distance.vincenty(coords_1, coords_2).km

279.35290160386563

In [ ]:
major_city

,major_town_and_city,All_Ages,Aged_16+,Aged_16_to_17,Aged_18_to_24,Aged_25_to_49,Aged_50_to_64,Aged_65+,town_name
0,Barnsley,"99,642","81,407","2,040","6,935","33,166","20,216","19,050","Barnsley, United Kingdom"
1,Basildon,"118,145","91,267","2,864","8,915","40,997","21,173","17,318","Basildon, United Kingdom"
2,Basingstoke,"115,111","91,318","2,465","8,376","41,671","21,732","17,074","Basingstoke, United Kingdom"
3,Bath,"105,730","90,154","1,971","21,848","31,911","16,996","17,428","Bath, United Kingdom"
4,Bedford,"93,767","74,104","2,376","7,449","31,695","17,197","15,387","Bedford, United Kingdom"
...,...,...,...,...,...,...,...,...,...
107,Worthing,"115,319","95,343","2,329","7,126","34,518","23,991","27,379","Worthing, United Kingdom"
108,York,"164,934","139,946","2,858","27,811","55,435","27,290","26,552","York, United Kingdom"
109,Cardiff,"356,425","290,560","7,148","52,783","123,170","56,677","50,782","Cardiff, United Kingdom"
110,Newport,"137,563","108,939","3,187","11,275","46,496","25,466","22,515","Newport, United Kingdom"


In [ ]:
consolidated=pd.merge(
    major_city, 
    gb_auth, 
    how="left", 
    left_on="major_town_and_city", 
    right_on="city")

consolidated_R=pd.merge(
    gb_auth,
    major_city,
    how="left",
    left_on="city",
    right_on="major_town_and_city")

In [ ]:
consolidated['city'].isnull().sum()
consolidated_R['major_town_and_city'].isnull().sum()

2580

In [ ]:
consolidated_R.shape

(2680, 17)

In [ ]:
consolidated[consolidated['city'].isna()]

,major_town_and_city,All_Ages,Aged_16+,Aged_16_to_17,Aged_18_to_24,Aged_25_to_49,Aged_50_to_64,Aged_65+,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
13,Bradford,"358,573","266,489","10,563","34,366","121,823","56,477","43,260",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Brighton and Hove,"245,504","208,437","4,700","36,429","94,495","41,953","30,860",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Cambridge,"149,155","122,662","3,006","27,819","48,468","22,496","20,873",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Chesterfield,"89,466","74,955","1,637","6,141","27,583","19,714","19,880",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Gloucester,"148,167","118,395","3,208","12,339","48,736","28,799","25,313",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Harlow,"87,420","67,450","1,923","6,001","30,039","16,040","13,447",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Newcastle upon Tyne,"290,688","241,270","5,530","52,498","96,810","45,083","41,349",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Newcastle-under-Lyme,"79,527","65,728","1,654","7,015","26,001","15,362","15,696",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Northampton,"230,070","179,380","5,056","19,459","77,985","40,932","35,948",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Peterborough,"179,349","136,618","4,093","12,250","64,325","29,898","26,052",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gb_auth[gb_auth["city"]=="St Alban"]

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper


In [ ]:

geolocator = Nominatim(user_agent="PersonalProject")
location = geolocator.geocode("London, United Kingdom")

print((location.latitude, location.longitude ))

(51.5073219, -0.1276474)


In [ ]:
#


temp_list = local_auth["local_authority"]
temp_long = []

for city in temp_list:
  temp_long.append(gb_auth[gb_auth["city"]==city]["lng"])




In [ ]:
consolidated.shape

(378, 17)

In [ ]:
pd.to_numeric(temp_long[0])

78   -1.5526
Name: lng, dtype: float64

## Use GeoNames to get location

In [ ]:
response = requests.get("http://api.geonames.org/search?q=london&username=personalproj2811")

response

<Response [200]>

In [ ]:
map_1 = KeplerGl()
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl()